## Importing packages

In [43]:
from nltk import word_tokenize, sent_tokenize
from string import punctuation
from sklearn.model_selection import train_test_split
from nltk.util import ngrams
from math import log, exp
from operator import itemgetter
import numpy as np

## Parsing into sentences and removing unnecessary symbols

In [44]:
#Opening File and tokenizing into sentences
text_file = open('Alice_in_Wonderland.txt','r').read().decode('utf-8')
sents_current = sent_tokenize(text_file)
sents = []
vocab = {}
translate_table = {}
for char in punctuation:
    if char == '-':
        translate_table[ord(char)] = u"\u0020"
    else:
        translate_table[ord(char)] = None

#Removing punctuation and making vocabulary
for sent in sents_current:
    sent = sent.lower()
    sent = sent.translate(translate_table) 
    words = word_tokenize(sent)
    msent = ['<s>']
    for word in words:
        if len(set(word).intersection(punctuation)) == 0 and (len(word)>1 or word == 'a' or word == 'i' or word == 'o') :
            msent.append(word)
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    msent.append('</s>')
    fsent = " ".join(msent)
    sents.append(fsent)
n_sents = len(sents)
vocab['<s>'] = 2*n_sents
vocab['</s>'] = 2*n_sents

tokens = sum(vocab.values())
types = len(vocab)
print 'Number of tokens = ' + str(tokens)
print 'Number of types = ' + str(types)
print 'Number of sentences = ' + str(n_sents)

Number of tokens = 30713
Number of types = 2560
Number of sentences = 976


## Diving dataset into 80% train and 20% test

In [45]:
train, test = train_test_split(sents, test_size=0.20, random_state=42)
print "Ratio of train and test data size = " + str(len(train)*1.0/len(test))

Ratio of train and test data size = 3.97959183673


## MLE of Unigrams

In [46]:
#Calculating MLE
uni_mle = {}
for i in vocab.keys():
    uni_mle[i] = (vocab[i]*1.0)/tokens
    
#Printing MLE for 10 unigrams
for i in uni_mle.keys()[:10]:
    print i, uni_mle[i] 

secondly 6.51190049816e-05
pardon 0.000195357014945
saves 3.25595024908e-05
knelt 3.25595024908e-05
four 0.000260476019926
sleep 0.000195357014945
hanging 9.76785074724e-05
ringlets 6.51190049816e-05
oldest 3.25595024908e-05
hate 6.51190049816e-05


## MLE of Bigrams

In [47]:
#Calculating MLE
bigrams = {}
bi_mle = {}
for sent in sents:
    ssent = sent.split()
    for i in range(0,len(ssent)-1):
        k = (ssent[i], ssent[i+1])
        try:
            bigrams[k] += 1
        except KeyError:
            bigrams[k] = 1
for i in bigrams.keys():
    bi_mle[i] = (bigrams[i]*1.0)/vocab[i[0]]

#Printing MLE for 10 biigrams
for i in bi_mle.keys()[:10]:
    print i, bi_mle[i]

(u'at', u'ours') 0.00471698113208
(u'fish', u'came') 0.125
(u'you', u'have') 0.0121654501217
(u'keep', u'herself') 0.0909090909091
(u'with', u'great') 0.0165745856354
(u'dodo', u'in') 0.0769230769231
(u'hatter', u'said') 0.0357142857143
(u'rabbit', u'was') 0.0392156862745
(u'claws', u'and') 1.0
(u'the', u'three') 0.00426309378806


## MLE of Trigrams

In [48]:
#Calculating MLE
trigrams = {}
tri_mle = {}
for sent in sents:
    ssent = sent.split()
    for i in range(0,len(ssent)-2):
        k = (ssent[i], ssent[i+1], ssent[i+2])
        try:
            trigrams[k] += 1
        except KeyError:
            trigrams[k] = 1
for i in trigrams.keys():
    tri_mle[i] = (trigrams[i]*1.0)/bigrams[(i[0], i[1])]

#Printing MLE for 10 trigrams
for i in tri_mle.keys()[:10]:
    print i, tri_mle[i]

(u'at', u'the', u'beginning') 0.0166666666667
(u'a', u'friend', u'of') 1.0
(u'alice', u'was', u'soon') 0.0588235294118
(u'heard', u'of', u'one') 0.25
(u'swallow', u'a', u'morsel') 1.0
(u'at', u'it', u'uneasily') 0.142857142857
(u'if', u'it', u'likes') 0.1
(u'you', u'could', u'only') 0.2
(u'house', u'opened', u'and') 1.0
(u'she', u'remembered', u'that') 0.2


## MLE of Quadrams

In [49]:
#Calculating MLE
quadgrams = {}
quad_mle = {}
for sent in sents:
    ssent = sent.split()
    for i in range(0,len(ssent)-3):
        k = (ssent[i], ssent[i+1], ssent[i+2], ssent[i+3])
        try:
            quadgrams[k] += 1
        except KeyError:
            quadgrams[k] = 1
for i in quadgrams.keys():
    quad_mle[i] = (quadgrams[i]*1.0)/trigrams[(i[0], i[1], i[2])]

#Printing MLE for 10 quadgrams
for i in quad_mle.keys()[:10]:
    print i, quad_mle[i]

(u'if', u'i', u'would', u'talk') 1.0
(u'what', u'a', u'funny', u'watch') 1.0
(u'hand', u'bit', u'to', u'try') 1.0
(u'<s>', u'therefore', u'i', u'mad') 1.0
(u'the', u'song', u'i', u'have') 1.0
(u'me', u'hear', u'the', u'name') 1.0
(u'which', u'word', u'sounded', u'best') 1.0
(u'in', u'chorus', u'yes', u'please') 1.0
(u'happens', u'and', u'if', u'i') 1.0
(u'and', u'under', u'sentence', u'of') 1.0


## Number of possible n-grams and number of present n-grams 

In [50]:
def ngram(n):
    ngram_arr = []
    poss_ngrams = types**n
    for sent in sents:
        ngram_arr += list(ngrams(sent.split(), n))
    present_ngrams = len(set(ngram_arr))
    return poss_ngrams, present_ngrams

#Printing
n = input('Enter a number:')
poss, present = ngram(n)
if n == 1:
    print "Number of possible unigrams = " + str(poss)
    print "Number of present unigrams = " + str(present)
elif n == 2:
    print "Number of possible bigrams = " + str(poss)
    print "Number of present bigrams = " + str(present)
elif n == 3:
    print "Number of possible trigrams = " + str(poss)
    print "Number of present trigrams = " + str(present)
elif n == 4:
    print "Number of possible quadgrams = " + str(poss)
    print "Number of present quadgrams = " + str(present)
else:
    print "Number of possible " + str(n) + "-grams = " + str(poss)
    print "Number of present " + str(n) + "-grams = " + str(present)

Enter a number:13
Number of possible 13-grams = 202824096036516704239472512860160000000000000
Number of present 13-grams = 17941


## Function to generate a sentence using a given ngram model

In [51]:
def generate(n):
    n = int(n) #n should be an integer
    
    #n should not be less than one
    if n<1:
        print "Error: n should be greater than or equal to one"
        return
    
    elif n>264:
        print "Error: too long sentences required. Corpus does not have this long sentences"
        return
    #for unigram model
    elif n == 1:
        s = ['<s>']
        gen_unigram = []
        gen_prob = []
        for i in uni_mle.keys():
            gen_unigram.append(i)
            gen_prob.append(uni_mle[i])
        norm_prob = np.divide(gen_prob, sum(gen_prob))
        while(s[-1] != '</s>' and len(s)<25):
            a = np.random.multinomial(1, norm_prob, size=1).tolist()
            index = a[0].index(1)
            s.append(gen_unigram[index])
    
    #for ngram model where n>1
    else:
        #making list of ngrams and (n-1)grams
        ngram_dict = {}
        ngraml_dict = {}
        for sent in sents:
            ngram_arr_n = list(ngrams(sent.split(), n))
            ngram_arr_nl = list(ngrams(sent.split(), n-1))
            for i in ngram_arr_n:
                try:
                    ngram_dict[i] += 1
                except KeyError:
                    ngram_dict[i] = 1 
            for i in ngram_arr_nl:
                try:
                    ngraml_dict[i] += 1
                except KeyError:
                    ngraml_dict[i] = 1 
                    
        #calculating MLE for ngram model
        n_mle = {}
        for i in ngram_dict.keys():
            li = []
            for j in range(n-1):
                li.append(i[j])
            n_mle[i] = (ngram_dict[i]*1.0)/ngraml_dict[tuple(li)]
        
        #selecting first ngram of sentence according to multinomial distribution
        s = []
        gen_ngram = []
        gen_prob = []
        for i in ngram_dict.keys():
            if i[0] == '<s>':
                gen_ngram.append(i)
                gen_prob.append(n_mle[i])
        norm_prob = np.divide(gen_prob, sum(gen_prob))
        a = np.random.multinomial(1, norm_prob, size=1).tolist()
        index = a[0].index(1)
        for i in gen_ngram[index]:
            s.append(i)
        
        #generating the rest of the sentence
        while(s[-1] != '</s>' and len(s)<25):
            gen_ngram = []
            gen_prob = []
            for b in ngram_dict.keys(): 
                if(list(b[:-1]) == s[-n+1:]):
                    gen_ngram.append(b)
                    gen_prob.append(n_mle[b])
            norm_prob = np.divide(gen_prob, sum(gen_prob))
            a = np.random.multinomial(1, norm_prob, size=1).tolist()
            index = a[0].index(1)
            s.append(gen_ngram[index][-1])
    print ' '.join(s)

n = input("Enter value of n for n-gram:")
for i in range(8):
    generate(n)
    

Enter value of n for n-gram:4
<s> but at any rate i ll never go there again said alice as she stood watching them and he checked himself suddenly the others
<s> she took down a jar from one of the legs of the table </s>
<s> by the bye what became of the baby said the cat or you wouldn have come here alice didn think that proved it at
<s> chapter advice from a caterpillar the caterpillar and alice looked very anxiously into its face in some alarm </s>
<s> presently the rabbit came near her she began in a voice of thunder and people began running about in all directions tumbling up against
<s> i didn know it was your table said alice it very interesting </s>
<s> come that finished the guinea pigs thought alice </s>
<s> would you like the mock turtle </s>


## Probability of a sentence given an ngram model

In [52]:
def probability(sentence, n):
    #unigram model
    if n == 1:
        p = 0
        for i in sentence.split():
            p += log(uni_mle[i])
            
    #bigram model
    elif n == 2:
        p = 0
        ssent = sentence.split()
        for i in range(1,len(ssent)):
            p += log(bi_mle[(ssent[i-1], ssent[i])])
            
    #trigram model
    elif n == 3:
        p=0
        ssent = sentence.split()
        for i in range(2,len(ssent)):
            p += log(tri_mle[(ssent[i-2], ssent[i-1], ssent[i])])
            
    #quadgram model
    elif n == 4:
        p=0
        ssent = sentence.split()
        for i in range(3,len(ssent)):
            p += log(quad_mle[(ssent[i-3], ssent[i-2], ssent[i-1], ssent[i])])
            
    #ngram model
    else:
        ngram_dict = {}
        ngraml_dict = {}
        p=0
        ssent = sentence.split()
        for sent in sents:
            #finding ngrams and (n-1)grams with their frequencies
            ngram_arr_n = list(ngrams(sent.split(), n))
            ngram_arr_nl = list(ngrams(sent.split(), n-1))
            for i in ngram_arr_n:
                try:
                    ngram_dict[i] += 1
                except KeyError:
                    ngram_dict[i] = 1 
            for i in ngram_arr_nl:
                try:
                    ngraml_dict[i] += 1
                except KeyError:
                    ngraml_dict[i] = 1 
                    
        #calculating log probability
        for i in range(n-1,len(ssent)):
            l=[]
            for j in range(i-n+1, i+1):
                l.append(ssent[j])
            t = tuple(l)
            tl = tuple(l[:-1])
            try:
                p += log(ngram_dict[t]) - log(ngraml_dict[tl])
            except:
                p = -99999999999999999999999999999999999999999999999
    return p

sentence = '<s> stop this moment i tell you but she went on all the same shedding gallons of tears until there was a large pool all round her about four inches deep and reaching half down the hall </s>'
n = input("Enter n for ngram model:")
prob = probability(sentence, n)
print 'Log of probability is:' + str(prob)
print 'Probability is:' + str(exp(prob))


Enter n for ngram model:5
Log of probability is:-0.69314718056
Probability is:0.5


### Add-One smoothing for bigram model

In [53]:
bia1_mle = {}
bigramsa1 = {}
diff = {}
for i in bigrams.keys():
    bia1_mle[i] = (bigrams[i] + 1.0)/(vocab[i[0]] + types)
    bigramsa1[i] = bia1_mle[i]*vocab[i[0]]
    diff[i] = bigrams[i] - bigramsa1[i]
sorted_diff = sorted(diff.items(), key=itemgetter(1))[::-1]
print "Bigrams, the change in count, original count, effective count after add-one smoothing for top 10 drastic changes are:"
for k in range(10):
    print sorted_diff[k][0], sorted_diff[k][1], bigrams[sorted_diff[k][0]], bigramsa1[sorted_diff[k][0]] 

#We can use bia1_mle instead of bi_mle in the probability function to compute probability of a sentence according to add 
#one smoothing on bigram model

Bigrams, the change in count, original count, effective count after add-one smoothing for top 10 drastic changes are:
(u'said', u'the') 176.895433488 209 32.1045665122
(u'of', u'the') 110.594014314 133 22.4059856864
(u'said', u'alice') 98.113170086 116 17.886829914
(u'in', u'a') 84.6830601093 97 12.3169398907
(u'in', u'the') 68.9453551913 79 10.0546448087
(u'<s>', u'i') 66.5177304965 118 51.4822695035
(u'it', u'was') 61.4786053883 76 14.5213946117
(u'and', u'the') 60.9114219114 82 21.0885780886
(u'at', u'the') 55.3347763348 60 4.66522366522
(u'as', u'she') 55.22387531 61 5.77612469005


There is a drastic change because we distribute the probability mass distribution to the bigrams that we have not seen. Also the change is maximum for the bigrams with highest frquency because maximum probability mass is taken from them in this case. 

## Good Turing Smoothing

In [54]:
def GTS(n):
    ngram_dict = {}
    for sent in sents:
        ngram_arr_n = list(ngrams(sent.split(), n))
        for i in ngram_arr_n:
            try:
                ngram_dict[i] += 1
            except:
                ngram_dict[i] = 1 
    m = max(ngram_dict.values())
    freqList = [0]*(m+1)
    for j in ngram_dict.keys():
        c = ngram_dict[j]
        freqList[c] +=1
    freqList[0] = sum(ngram_dict.values())
    
    #Taking top 10 counts
    print "Differences for " + str(n) + "-gram model are: -----"
    diff = []
    for i in range(1, min(m,11)):
        if freqList[i] == 0:
            break
        else:
            diff.append(i - ((i + 1.0)*freqList[i+1])/freqList[i])
    print diff
    mean = np.mean(diff)
    print "Mean:" + str(mean) + "\n"
    return mean

mean = []
for i in range(1,6):
    m = GTS(i)
    mean.append(m)
print "d:" +str(np.mean(mean))

Differences for 1-gram model are: -----
[0.2827461607949413, 0.2770780856423174, 0.47368421052631593, 0.8402777777777777, 0.9780219780219781, -1.0, -0.21311475409836067, 2.2727272727272725, -1.2857142857142865, 2.666666666666667]
Mean:0.5292373112344623

Differences for 2-gram model are: -----
[0.6722103502740872, 0.8231292517006803, 1.2890173410404624, 0.4020270270270272, 1.056338028169014, 0.75, 0.980952380952381, 1.3924050632911396, 2.275862068965517, 0.4102564102564106]
Mean:1.005219792167672

Differences for 3-gram model are: -----
[0.8549446241301578, 1.1972972972972973, 1.404040404040404, 1.689873417721519, 0.06849315068493134, 3.6666666666666665, 2.2, 0.5, 3.0, 2.666666666666667]
Mean:1.7247982227207643

Differences for 4-gram model are: -----
[0.9441242058316153, 1.4984939759036144, 1.5225225225225225, 2.5365853658536586, 1.0, 4.25, -9.0, 5.75, -11.0, 4.5]
Mean:0.20017260701114098

Differences for 5-gram model are: -----
[0.9782248716674946, 1.6007604562737643, 2.0857142857142

D values fluctuates for different models. However it is somewhat constant in a single model for different counts. Also, it is sometomes negative and sometimes positive. Mean d:0.66

## Perplexity of add one smoothing and good turing smoothing for bigram model

In [56]:
#vocab for train
vocab_train = {}
for sent in train:
    ssent = sent.split()
    for i in ssent:
        try:
            vocab_train[i]+=1
        except KeyError:
            vocab_train[i] = 1 
vocab_types = len(vocab_train.keys())

#bigrams for train set
bigrams_train = {}
bi_mle_train = {}
for sent in train:
    ssent = sent.split()
    for i in range(0,len(ssent)-1):
        k = (ssent[i], ssent[i+1])
        try:
            bigrams_train[k] += 1
        except KeyError:
            bigrams_train[k] = 1
for i in bigrams_train.keys():
    bi_mle_train[i] = (bigrams_train[i]*1.0)/vocab_train[i[0]]

#add-one soothing on trainset
bia1_mle_train = {}
for i in bigrams_train.keys():
    bia1_mle_train[i] = (bigrams_train[i] + 1.0)/(vocab_train[i[0]] + vocab_types)

#calculating perplexity of add one smoothing
a1p = []
for j in test:
    p=0
    ssent = j.split()
    for i in range(1, len(ssent)):
        try:
            p = p + log(1.0/bia1_mle_train[(ssent[i-1], ssent[i])])
        except KeyError:
            try:
                p = p + log((vocab_train[ssent[i-1]] + vocab_types))    #bia1_mle_train['<unk>'] = 1.0/(vocab_train[ssent[i-1]] + vocab_types)
            except KeyError:
                p = p + log(1+vocab_types)              #vocab_train[ssent[i-1]] = 1
    a1p.append(exp(p*(1.0/(len(ssent)))))
print "Perplexity of add one smoothing: " + str(np.mean(a1p))

#calculating perplexity for good turing smoothing
m = max(bigrams_train.values())
freqList = [0]*(m+1)
for j in bigrams_train.keys():
    c = bigrams_train[j]
    freqList[c] +=1
freqList[0] = sum(bigrams_train.values())

#We use the value of d from the previous part to calculate the effective counts. 
d = 0.8786209670155173 #d for bigram model
bigts_mle_train = {}
for i in bigrams_train.keys():
    bigts_mle_train[i] = (bigrams_train[i] - d)/(vocab_train[i[0]])
    
#Number of unigrams with unit frequency 
n_uni = 0
for i in vocab_train.values():
    if i==1:
        n_uni +=1
    
#calculating perplexity of good turing smoothing
gtsp = []
for j in test:
    p=0
    ssent = j.split()
    for i in range(1, len(ssent)):
        try:
            p = p + log(1.0/bigts_mle_train[(ssent[i-1], ssent[i])])
        except KeyError:
            try:
                p = p + log((freqList[0]*vocab_train[ssent[i-1]])/freqList[1])    #bia1_mle_train['<unk>'] = c*/(vocab_train[ssent[i-1]])
            except KeyError:
                m = sum(vocab_train.values())
                p = p + log((freqList[0]*n_uni*1.0)/(freqList[1]*m))    #vocab_train[ssent[i-1]] = n_uni/sum(vocab.values())
    gtsp.append(exp(p*(1.0/(len(ssent)))))
print "Perplexity of good turing smoothing: " + str(np.mean(gtsp))
print "Good turing is better by " + str(np.mean(a1p)/np.mean(gtsp)) + ' times'

Perplexity of add one smoothing: 570.2244425739852
Perplexity of good turing smoothing: 32.66473807037255
Good turing is better by 17.45688091377007 times


The perplexity of good turing smoothing is less than the perplexity of add one smoothing. Thus, it is better.